<a href="https://colab.research.google.com/github/rebekah-eichberg/playpause/blob/main/fine_tuning_attempt_rebekah.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import os
# os.kill(os.getpid(), 9)

In [ ]:
import bitsandbytes as bnb
print("bitsandbytes version:", bnb.__version__)

bitsandbytes version: 0.46.1


In [ ]:
# Login into Hugging Face Hub
from huggingface_hub import notebook_login
notebook_login()

### Install Python packages

Install the Hugging Face libraries required for running the Gemma model and making requests.

In [ ]:
# Install a transformers version that supports Gemma 3n (>= 4.53)
# !pip install "transformers>=4.53.0" "timm>=1.0.16"

## Define formatting helper functions

Create a chat helper to manage and display the conversations.

In [ ]:
import torch

GEMMA_PATH = "google/gemma-3n-E4B-it" #@param ["google/gemma-3n-E2B-it", "google/gemma-3n-E4B-it"]
RESOURCE_URL_PREFIX = "https://raw.githubusercontent.com/google-gemini/gemma-cookbook/refs/heads/main/Demos/sample-data/"

from IPython.display import Audio, Image, Markdown, display

class ChatState():
  def __init__(self, model, processor):
    self.model = model
    self.processor = processor
    self.history = []

  def send_message(self, message, max_tokens=256):
    self.history.append(message)

    input_ids = self.processor.apply_chat_template(
        self.history,
        add_generation_prompt=True,
        tokenize=True,
        return_dict=True,
        return_tensors="pt",
    )
    input_len = input_ids["input_ids"].shape[-1]

    input_ids = input_ids.to(self.model.device, dtype=model.dtype)
    outputs = self.model.generate(
        **input_ids,
        max_new_tokens=max_tokens,
        disable_compile=True
    )
    text = self.processor.batch_decode(
        outputs[:, input_len:],
        skip_special_tokens=True,
        clean_up_tokenization_spaces=True
    )
    self.history.append({
        "role": "assistant",
        "content": [
            {"type": "text", "text": text[0]},
        ]
    })

    # display chat
    for item in message['content']:
      if item['type'] == 'text':
        formatted_prompt = "<font size='+1' color='brown'>🙋‍♂️<blockquote>\n" + item['text'] + "\n</blockquote></font>"
        display(Markdown(formatted_prompt))
      elif item['type'] == 'audio':
        display(Audio(item['audio']))
      elif item['type'] == 'image':
        display(Image(item['image']))

    formatted_text = "<font size='+1' color='teal'>🤖<blockquote>\n" + text[0] + "\n</blockquote></font>"
    display(Markdown(formatted_text))


## Load Model

In [ ]:
!pip install trl

In [ ]:
from trl import SFTTrainer
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    AutoProcessor,
    AutoModelForImageTextToText,
    TrainingArguments,
    BitsAndBytesConfig,
    DataCollatorForLanguageModeling
)
from datasets import load_dataset
import torch

processor = AutoProcessor.from_pretrained(GEMMA_PATH)
model = AutoModelForImageTextToText.from_pretrained(GEMMA_PATH, torch_dtype="auto", device_map="auto")

print(f"Device: {model.device}")
print(f"DType: {model.dtype}")

processor_config.json:   0%|          | 0.00/98.0 [00:00<?, ?B/s]

chat_template.jinja:   0%|          | 0.00/1.63k [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.20M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.70M [00:00<?, ?B/s]

...

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/3.08G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/2.66G [00:00<?, ?B/s]

In [ ]:
!pip install python-docx

In [ ]:
from docx import Document
import json
import re

system_prompt = (
    "You are a cheerful and friendly tutor for children aged 2 to 4. "
    "Use simple words and fun metaphors to explain things clearly. "
    "Be playful and keep answers short and exciting. Maybe use known cartoon characters in your explanation."
    "It needs to be fun and engaging."
)

# Load DOCX file
doc = Document("Ages 2-4.docx")

# Initialize storage
questions = []
answers = []
wow_facts = []
categories = []

chat_entries = []
instruction_entries = []

current_category = None
question_pattern = re.compile(r'^[A-Z].*\?$')
sparky_prefix = "Sparky's Answer: "
wow_prefix = "Wow! Fact: "

# Parse document
i = 0
while i < len(doc.paragraphs):
    text = doc.paragraphs[i].text.strip()

    if not text:
        i += 1
        continue

    # Update current category
    if text in [
        "Human Body & Health", "Animals & Nature", "Earth & Space",
        "Physics & Chemistry", "Technology"
    ]:
        current_category = text
        i += 1
        continue

    # Detect questions
    if question_pattern.match(text):
        question = text
        answer = ""
        wow_fact = ""

        # Look ahead for answer and wow fact
        j = i + 1
        while j < len(doc.paragraphs):
            next_text = doc.paragraphs[j].text.strip()

            if next_text.startswith(sparky_prefix):
                answer = next_text[len(sparky_prefix):].strip()
            elif next_text.startswith(wow_prefix):
                wow_fact = next_text[len(wow_prefix):].strip()
                break
            j += 1

        if answer and wow_fact:
            full_response = f"{answer}\n{wow_fact}"

            # Save to both formats
            # 1. Chat format
            chat_entry = {
                "messages": [
                    {
                        "role": "system",
                        "content": [{"type": "text", "text": "You are Sparky, a fun robot who gives kid-friendly explanations with simple metaphors and a 'Wow! Fact!' at the end."}]
                    },
                    {
                        "role": "user",
                        "content": [{"type": "text", "text": question}]
                    },
                    {
                        "role": "assistant",
                        "content": [{"type": "text", "text": full_response}]
                    }
                ]
            }

            # 2. Instruction format
            instruction_entry = {
                "instruction": question,
                "input": f"Category: {current_category}",
                "output": full_response
            }

            chat_entries.append(chat_entry)
            instruction_entries.append(instruction_entry)

        i = j  # move past the wow fact
    else:
        i += 1

# Save both files
with open("chat_format.json", "w", encoding="utf-8") as f:
    json.dump(chat_entries, f, indent=2, ensure_ascii=False)

with open("instruction_format.json", "w", encoding="utf-8") as f:
    json.dump(instruction_entries, f, indent=2, ensure_ascii=False)

print(f"Processed {len(chat_entries)} Q&A pairs")


In [ ]:
from trl import SFTTrainer
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    TrainingArguments,
    BitsAndBytesConfig,
    DataCollatorForLanguageModeling
)
from peft import LoraConfig, get_peft_model
from datasets import load_dataset
import torch

In [ ]:
# Load dataset
dataset = load_dataset("json", data_files="chat_format.json", split="train")

In [ ]:
!pip install -q transformers peft accelerate datasets bitsandbytes trl

In [ ]:

print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("Device:", torch.cuda.get_device_name(0))

In [ ]:
# !pip uninstall bitsandbytes -y
# !pip install -U bitsandbytes


In [ ]:
model_name = GEMMA_PATH

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype="bfloat16",
    # llm_int8_enable_fp32_cpu_offload=True # Removed this line
)

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    "google/gemma-3n-E2B-it",
    device_map="auto",
    torch_dtype="auto"
)

tokenizer = AutoTokenizer.from_pretrained("google/gemma-3n-E2B-it")


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
# LoRA configuration
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],  # Use correct target modules for Gemma
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

In [ ]:
model = get_peft_model(model, lora_config)

ImportError: Using `bitsandbytes` 4-bit quantization requires the latest version of bitsandbytes: `pip install -U bitsandbytes`

In [ ]:
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

In [ ]:
training_args = TrainingArguments(
    output_dir="./gemma-qlora",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    num_train_epochs=3,
    save_strategy="epoch",
    logging_steps=10,
    learning_rate=2e-4,
    fp16=True,
    max_steps=100,  # test run
)

In [ ]:
def formatting_func(example):
    text = ""
    for msg in example["messages"]:
        role = msg["role"]
        content = msg["content"].strip()
        if role == "system":
            text += f"<|system|>\n{content}\n"
        elif role == "user":
            text += f"<|user|>\n{content}\n"
        elif role == "assistant":
            text += f"<|assistant|>\n{content}\n"
    return text

In [ ]:
trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    formatting_func=formatting_func,
    data_collator=data_collator,
)

Applying formatting function to train dataset:   0%|          | 0/37 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/37 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/37 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Currently logged in as: rebekah-eichberg4 (rebekah-eichberg4-kaggle) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Step,Training Loss
10,84.500100
20,81.101700
30,19.131200
40,10.598300
50,8.052500
60,6.578500
70,5.400500
80,4.430200
90,3.543000
100,2.818600


wandb: WARNING The get_url method is deprecated and will be removed in a future release. Please use `run.url` instead.


TrainOutput(global_step=100, training_loss=22.615460300445555, metrics={'train_runtime': 602.1152, 'train_samples_per_second': 1.329, 'train_steps_per_second': 0.166, 'total_flos': 1882412908623360.0, 'train_loss': 22.615460300445555})

In [ ]:
trainer.save_model()

In [ ]:
# free the memory again
del model
del trainer
torch.cuda.empty_cache()

In [ ]:
import torch
from transformers import pipeline

model_id = "gemma-qlora-lora"

# Load Model with PEFT adapter
model = AutoModelForCausalLM.from_pretrained(
  model_id,
  device_map="auto",
  torch_dtype=torch.bfloat16,
  attn_implementation="eager",
)
tokenizer = AutoTokenizer.from_pretrained(model_id)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:2397: UserWarning: for model.language_model.layers.0.self_attn.q_proj.lora_A.default.weight: copying from a non-meta parameter in the checkpoint to a meta parameter in the current model, which is a no-op. (Did you mean to pass `assign=True` to assign items in the state dictionary to their corresponding key in the module instead of copying them in place?)
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:2397: UserWarning: for model.language_model.layers.0.self_attn.q_proj.lora_B.default.weight: copying from a non-meta parameter in the checkpoint to a meta parameter in the current model, which is a no-op. (Did you mean to pass `assign=True` to assign items in the state dictionary to their corresponding key in the module instead of copying them in place?)
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:2397: UserWarning: for model.language_model.layers.

ValueError: We need an `offload_dir` to dispatch this model according to this `device_map`, the following submodules need to be offloaded: model.language_model.layers.2, model.language_model.layers.3, model.language_model.layers.4, model.language_model.layers.5, model.language_model.layers.6, model.language_model.layers.7, model.language_model.layers.8, model.language_model.layers.9, model.language_model.layers.10, model.language_model.layers.11, model.language_model.layers.12, model.language_model.layers.13, model.language_model.layers.14, model.language_model.layers.15, model.language_model.layers.16, model.language_model.layers.17, model.language_model.layers.18, model.language_model.layers.19, model.language_model.layers.20, model.language_model.layers.21, model.language_model.layers.22, model.language_model.layers.23, model.language_model.layers.24, model.language_model.layers.25, model.language_model.layers.26, model.language_model.layers.27, model.language_model.layers.28, model

In [ ]:
trainer.model.save_pretrained("gemma-qlora-lora")
tokenizer.save_pretrained("gemma-qlora-lora")


('gemma-qlora-lora/tokenizer_config.json',
 'gemma-qlora-lora/special_tokens_map.json',
 'gemma-qlora-lora/chat_template.jinja',
 'gemma-qlora-lora/tokenizer.model',
 'gemma-qlora-lora/added_tokens.json',
 'gemma-qlora-lora/tokenizer.json')

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel
import torch

base_model_name = GEMMA_PATH  # or your base checkpoint

# Load base model with the same quantization config used during training
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    quantization_config=bnb_config, # Use quantization config
    device_map="auto"
)

# Load LoRA adapter
model = PeftModel.from_pretrained(base_model, "gemma-qlora-lora")

# Generate text
inputs = tokenizer("Why do I have to wash my hands?", return_tensors="pt").to(model.device)
outputs = model.generate(**inputs, max_new_tokens=100)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

ValueError: Some modules are dispatched on the CPU or the disk. Make sure you have enough GPU RAM to fit the quantized model. If you want to dispatch the model on the CPU or the disk while keeping these modules in 32-bit, you need to set `llm_int8_enable_fp32_cpu_offload=True` and pass a custom `device_map` to `from_pretrained`. Check https://huggingface.co/docs/transformers/main/en/main_classes/quantization#offload-between-cpu-and-gpu for more details. 

In [ ]:
!pip install -U bitsandbytes